# Chapter 3 Multiple Linear Regression
多元线性回归

## 导入相关库 

In [1]:
###Boston房价预测模型
#导入模块

%matplotlib notebook
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle


## 通过pandas导入数据 

In [2]:

#读取数据文件

df = pd.read_csv("F:\Jupyter/tensorflow/MLR/data/boston.csv",header = 0)

#显示数据摘要描述信息
print(df.describe())


             CRIM         ZN       INDUS         CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677082   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

              AGE         DIS         RAD         TAX     PTRATIO       LSTAT  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534   12.653063   
std     28.148861    2.1057

## 数据准备

In [3]:
#获取df的值

df = df.values
print(df)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 4.9800e+00 2.4000e+01]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 9.1400e+00 2.1600e+01]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 4.0300e+00 3.4700e+01]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 5.6400e+00 2.3900e+01]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 6.4800e+00 2.2000e+01]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 7.8800e+00 1.1900e+01]]


In [4]:
#把 values格式转换成numpy的数组

df = np.array(df)
print(df)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 4.9800e+00 2.4000e+01]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 9.1400e+00 2.1600e+01]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 4.0300e+00 3.4700e+01]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 5.6400e+00 2.3900e+01]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 6.4800e+00 2.2000e+01]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 7.8800e+00 1.1900e+01]]


In [6]:
# x_data 为前12列的特征数据
x_data = df[:,:12]

# y_data 为最后一列标签数据
y_data = df[:,12]

In [9]:
print(x_data,'\n shape=', x_data.shape)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 2.9600e+02 1.5300e+01 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 2.4200e+02 1.7800e+01 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 2.4200e+02 1.7800e+01 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.7300e+02 2.1000e+01 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.7300e+02 2.1000e+01 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.7300e+02 2.1000e+01 7.8800e+00]] shape= (506, 12)


In [10]:
print(y_data,'\n shape=',y_data.shape)

[24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 15.  18.9 21.7 20.4
 18.2 19.9 23.1 17.5 20.2 18.2 13.6 19.6 15.2 14.5 15.6 13.9 16.6 14.8
 18.4 21.  12.7 14.5 13.2 13.1 13.5 18.9 20.  21.  24.7 30.8 34.9 26.6
 25.3 24.7 21.2 19.3 20.  16.6 14.4 19.4 19.7 20.5 25.  23.4 18.9 35.4
 24.7 31.6 23.3 19.6 18.7 16.  22.2 25.  33.  23.5 19.4 22.  17.4 20.9
 24.2 21.7 22.8 23.4 24.1 21.4 20.  20.8 21.2 20.3 28.  23.9 24.8 22.9
 23.9 26.6 22.5 22.2 23.6 28.7 22.6 22.  22.9 25.  20.6 28.4 21.4 38.7
 43.8 33.2 27.5 26.5 18.6 19.3 20.1 19.5 19.5 20.4 19.8 19.4 21.7 22.8
 18.8 18.7 18.5 18.3 21.2 19.2 20.4 19.3 22.  20.3 20.5 17.3 18.8 21.4
 15.7 16.2 18.  14.3 19.2 19.6 23.  18.4 15.6 18.1 17.4 17.1 13.3 17.8
 14.  14.4 13.4 15.6 11.8 13.8 15.6 14.6 17.8 15.4 21.5 19.6 15.3 19.4
 17.  15.6 13.1 41.3 24.3 23.3 27.  50.  50.  50.  22.7 25.  50.  23.8
 23.8 22.3 17.4 19.1 23.1 23.6 22.6 29.4 23.2 24.6 29.9 37.2 39.8 36.2
 37.9 32.5 26.4 29.6 50.  32.  29.8 34.9 37.  30.5 36.4 31.1 29.1 50.
 33.3 3

## 模型建立

### 定义特征数据和标签数据的占位符

In [11]:
#12个特征数据（12列）None表示行数不定，列数为12

x = tf.placeholder(tf.float32,[None,12], name = "X")

#1个标签数据
y = tf.placeholder(tf.float32,[None,1], name = "Y")

### 定义模型函数

In [13]:
#定义一个命名空间

#给模型下的语句定义了一个命名空间，相当于打包在一起，计算图中会显示一个节点
with tf.name_scope("Model"):
    w = tf.Variable(tf.random_normal([12,1], stddev = 0.01), name="W")
    
    # b初始化值为1.0
    b = tf.Variable(1.0, name = "b")
    
    # w和x是矩阵相乘，用matmul，不能用*或multiply
    def model(x, w, b):
        return tf.matmul(x, w) + b
    
    # 预测计算操作， 前向计算节点
    pred = model(x, w, b)
    

## 3训练模型 

### 设置模型训练超参数 

In [14]:
train_epochs = 50

learning_rate = 0.01

### 定义损失函数&优化器

In [20]:
with tf.name_scope("LossFunction"):
    loss_function = tf.reduce_mean(tf.pow(pred - y,2))
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

In [21]:
###定义会话，初始化变量

sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

### 迭代训练

In [23]:
for epoch in range (train_epochs):
    loss_sum = 0.0
    for xs,ys in zip(x_data, y_data):
        xs = xs.reshape(1, 12)
        ys = ys.reshape(1,1)        #feed数据必须和placeholder的shape保持一致
        
        _, loss = sess.run([optimizer, loss_function], feed_dict={x:xs, y:ys})
        
        loss_sum = loss_sum + loss
        
    #打乱数据顺序
    x_data, y_data = shuffle(x_data, y_data)
    
    b0temp = b.eval(session=sess)
    w0temp = w.eval(session=sess)
    loss_average = loss_sum/len(y_data)
    print("epoch=",epoch+1,"loss=", loss_average, "b=", b0temp, "w=", w0temp)
    

epoch= 1 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 2 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 3 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 4 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 5 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 6 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 7 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 8 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
epoch= 9 loss= nan b= nan w= [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [n